In [ ]:
import requests
import pandas as pd
import csv

# Last.fm API credentials
lastfm_api_key = '340a0195516763a1af37aa4168461814'

def get_similar_song(track_name,artist_name)-> tuple:
	song_results =[]
	artist_results =[]

	# Define the Last.fm API endpoint and parameters
	lastfm_url = 'http://ws.audioscrobbler.com/2.0/'
	params = {
		'method': 'track.getsimilar',
		'track': track_name,  # Use the track name from Spotify data
		'artist': artist_name,  # Use the first artist's name
		'api_key': lastfm_api_key,
		'format': 'json',
		'autocorrect': 1,  # Enable auto-correction of spelling mistakes
		'limit': 10  # Limit the results to 10 similar tracks
	}

	# Make the API call to Last.fm
	lastfm_response = requests.get(lastfm_url, params=params)

	# Parse the response
	if lastfm_response.status_code == 200:
		similar_tracks_data = lastfm_response.json()
		similar_tracks = similar_tracks_data.get('similartracks', {}).get('track', [])
		for i, similar_track in enumerate(similar_tracks, start=1):
			print(f"{i}. {similar_track['name']} by {similar_track['artist']['name']}")	
			song_results.append({similar_track['name']})
			artist_results.append({similar_track['artist']['name']})
			#return similar_track['name'], similar_track['artist']['name']
		return song_results,artist_results
	else:
		print(lastfm_response.status_code)
		print(lastfm_response.json())
		return
#		print(f"Error: {lastfm_response.status_code} - {lastfm_response.text}")

In [ ]:

# Path to the directory containing your CSV files
file_path = 'data/combined.csv'
df = pd.read_csv(file_path,encoding='latin1') 

In [4]:
my_data = pd.read_csv("data/combined.csv") 

In [7]:
my_data.head()

,Track ID,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,Genres,...,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,4fSIb4hdOQ151TILNsSEaF,Todo De Ti,Todo De Ti,Rauw Alejandro,2021-05-20,199604.0,72.0,s86x8wq9jeob731cq3j2vla7k,2022-03-30T23:23:12Z,"reggaeton,urbano latino,trap latino,latin",...,3.0,-3.605,0.0,0.0506,0.3100,0.000163,0.0932,0.342,127.949,4.0
1,2TH65lNHgvLxCKXM3apjxI,Callaita,Callaita,"Bad Bunny,Tainy",2019-05-31,250533.0,74.0,kat.a.538,2022-03-30T00:23:48Z,"reggaeton,trap latino,latin,urbano latino",...,2.0,-4.773,1.0,0.3090,0.6000,0.000002,0.2430,0.244,176.169,4.0
2,0SqqAgdovOE24BzxIClpjw,Yo Perreo Sola,YHLQMDLG,Bad Bunny,2020-02-29,172127.0,72.0,kat.a.538,2022-03-30T00:23:00Z,"reggaeton,trap latino,latin,urbano latino",...,11.0,-5.160,1.0,0.0695,0.0210,0.000065,0.3440,0.453,97.014,4.0
3,2DEZmgHKAvm41k4J3R2E9Y,Safaera,YHLQMDLG,"Bad Bunny,Jowell & Randy,Ñengo Flow",2020-02-29,295176.0,75.0,kat.a.538,2022-03-30T00:23:11Z,"reggaeton,trap latino,latin,urbano latino",...,2.0,-4.074,1.0,0.3790,0.0103,0.000000,0.1070,0.685,96.172,4.0
4,4gChz67rTHfVzKCZAHsifM,La Tóxica (feat. Jay Wheeler & Tempo) - Remix,La Tóxica (feat. Jay Wheeler & Tempo) [Remix],"Farruko,Sech,Myke Towers,Jay Wheeler,Tempo",2020-10-21,294240.0,52.0,kat.a.538,2022-03-30T00:24:06Z,"reggaeton,trap latino,latin,urbano latino,lati...",...,9.0,-3.250,1.0,0.1040,0.1020,0.000001,0.0562,0.604,92.019,4.0


In [ ]:
import lyricsgenius
# Grab lyrics for similar song, to pass to the model
my_data_similar_df = pd.DataFrame(columns=["title", "artist", "lyrics", "similar_track", "similar_artist"])
title_list = []
artist_list = []
lyric_list=[]
similar_track_list = []
similar_artist_list = []

for i in range(0,len(my_data)):
    row = my_data.iloc[i]
    song_title = row["Track Name"]
    artist_name = row["Artist Name(s)"]
    print(f"finding {song_title}, {artist_name}")

    result = get_similar_song(song_title, artist_name)

    if result is not None:
        for r in result:
            print(r)
            similar_song, similar_artist = result
            try:
                song = genius.search_song(similar_song, similar_artist)
        
                if song:
                    print("got song")
                    new_row = pd.DataFrame([{
                    "title": song_title,
                    "artist": artist_name,
                    "lyrics": song.lyrics,
                    "similar_track": similar_song,
                    "similar_artist": similar_artist
                }])
                    my_data_similar_df = pd.concat([my_data_similar_df, new_row], ignore_index=True)

                # lyric_list.append(song.lyrics)
                # title_list.append(song_title)
                # artist_list.append(artist_name)
                # similar_track_list.append(row["title"])
                # similar_artist_list.append(row["artist"])
            except Exception as e:
                print(f"Error fetching lyrics for {similar_song} by {similar_artist}: {e}")
                continue
    else:
        continue


finding Todo De Ti, Rauw Alejandro
1. Sexo Virtual by Rauw Alejandro
Searching for "Sexo Virtual" by Rauw Alejandro...
Done.
got song
finding Callaita, Bad Bunny,Tainy
finding Yo Perreo Sola, Bad Bunny
1. La Difícil by Bad Bunny
Searching for "La Difícil" by Bad Bunny...


KeyboardInterrupt: 

In [ ]:
#Read a folder of several playlists

import os
import pandas as pd
#  Path to the folder containing the CSV files
folder_path = "/Users/0wner/Documents/Northeastern/Academic/S25/NLP/Final_Project/NLP_Final/daisy_playlist"

# List to store individual DataFrames
dfs = []

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):  # Adjust this if you're using other formats like .xlsx
        file_path = os.path.join(folder_path, filename)
        
        # Read each CSV into a DataFrame
        df = pd.read_csv(file_path)
        
        # Append the DataFrame to the list
        dfs.append(df)


#Concatenate the dfs
daisy_dfs = pd.DataFrame() 

for df in dfs:
    daisy_dfs = pd.concat([daisy_dfs, df], ignore_index=True)

In [ ]:
#Concatenate it to more playlists
kats_dfs = pd.read_csv("data/combined.csv") 
exportify_dfs = pd.concat([daisy_dfs, kats_dfs], ignore_index=True)


In [ ]:
#Extract numerical features
exportify_numerical_features = exportify_dfs[['Track Name', 'Artist Name(s)', 'Danceability','Energy', 'Speechiness','Acousticness','Instrumentalness','Liveness','Valence']]

In [11]:
exportify_numerical_features.to_csv("data/exportify_numerical_features.csv", index=False, encoding='utf-8')